In [82]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.util import ngrams 
from collections import Counter
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud,ImageColorGenerator
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
import string
from nltk.corpus import stopwords
stop = stopwords.words('english')

train = pd.read_csv(r'C:\datasets\competition\tweet sentiment\train_2kmZucJ.csv')
test = pd.read_csv(r'C:\datasets\competition\tweet sentiment\test_oJQbWVk.csv')

In [83]:
train.describe()

,id,label
count,7920.000000,7920.000000
mean,3960.500000,0.255808
std,2286.451399,0.436342
min,1.000000,0.000000
25%,1980.750000,0.000000
50%,3960.500000,0.000000
75%,5940.250000,1.000000
max,7920.000000,1.000000


In [84]:
test.describe()

,id
count,1953.000000
mean,8897.000000
std,563.926857
min,7921.000000
25%,8409.000000
50%,8897.000000
75%,9385.000000
max,9873.000000


In [85]:
combi = train.append(test,ignore_index=True)

C:\Users\320087401\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [86]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt  

In [87]:
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*")

In [88]:
# remove special characters, numbers, punctuations
combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

In [89]:
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


In [90]:
combi.describe()

,id,label
count,9873.000000,7920.000000
mean,4937.000000,0.255808
std,2850.233938,0.436342
min,1.000000,0.000000
25%,2469.000000,0.000000
50%,4937.000000,0.000000
75%,7405.000000,1.000000
max,9873.000000,1.000000


In [91]:
combi.isnull().sum()

id               0
label         1953
tweet            0
tidy_tweet       0
dtype: int64

In [92]:
combi['label'] = combi['label'].fillna(0)

In [93]:
combi.isnull().sum()
combi.label.describe()

count    9873.000000
mean        0.205206
std         0.403873
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [94]:
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [#fingerprint, #Pregnancy, Test, https, MfQV, ...
1    [Finally, transparant, silicon, case, Thanks, ...
2    [love, this, Would, #talk, #makememories, #unp...
3    [wired, know, George, made, that, #iphone, #cu...
4    [What, amazing, service, Apple, even, talk, ab...
Name: tidy_tweet, dtype: object

In [95]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0    [#fingerprint, #pregnanc, test, http, mfqv, #a...
1    [final, transpar, silicon, case, thank, uncl, ...
2    [love, thi, would, #talk, #makememori, #unplug...
3    [wire, know, georg, made, that, #iphon, #cute,...
4    [what, amaz, servic, appl, even, talk, about, ...
Name: tidy_tweet, dtype: object

In [96]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combi['tidy_tweet'] = tokenized_tweet

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1500, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(combi['tidy_tweet'])
X = bow.toarray()
y = combi['label']
X.shape
y.shape


(9873,)

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [99]:
# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# classifier.fit(X_train, y_train)

In [100]:
# y_pred = classifier.predict(X_test)

In [101]:
# from sklearn.metrics import accuracy_score
# print(accuracy_score(y_test,y_pred))

In [102]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [103]:
predictions = classifier.predict(X_test)
predictions

array([0., 1., 1., ..., 0., 0., 0.])

In [104]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.6850632911392405


In [117]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
y_pred
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))
f1_score(y_test,y_pred)

0.8491139240506329


0.6119791666666666

In [119]:
submission = pd.DataFrame({'id':test['id'],'label':pd.Series(y_pred)})

#Visualize the first 5 rows
submission.head()

,id,label
0,7921.0,0.0
1,7922.0,0.0
2,7923.0,1.0
3,7924.0,0.0
4,7925.0,0.0


In [120]:
filename = 'tweet_senti1.csv'

submission.to_csv(filename,index=False)